In [13]:
import stanza
import pandas as pd

In [65]:
stanza.download('pl', package='lfg')
nlp = stanza.Pipeline('pl', package='lfg')
sentiment_dict_file = 'nawl-analysis.csv'
sentiment_dict_file2 = 'test_sentiment_dictionary.csv'
sentiment_dictionary = pd.read_csv(sentiment_dict_file2)

2021-01-22 22:46:34 INFO: Downloading these customized packages for language: pl (Polish)...
| Processor | Package |
-----------------------
| tokenize  | lfg     |
| pos       | lfg     |
| lemma     | lfg     |
| depparse  | lfg     |
| pretrain  | lfg     |

2021-01-22 22:46:34 INFO: File exists: /home/jarczi/stanza_resources/pl/tokenize/lfg.pt.
2021-01-22 22:46:34 INFO: File exists: /home/jarczi/stanza_resources/pl/pos/lfg.pt.
2021-01-22 22:46:34 INFO: File exists: /home/jarczi/stanza_resources/pl/lemma/lfg.pt.
2021-01-22 22:46:34 INFO: File exists: /home/jarczi/stanza_resources/pl/depparse/lfg.pt.
2021-01-22 22:46:34 INFO: File exists: /home/jarczi/stanza_resources/pl/pretrain/lfg.pt.
2021-01-22 22:46:34 INFO: Finished downloading models and saved to /home/jarczi/stanza_resources.
2021-01-22 22:46:34 INFO: Loading these models for language: pl (Polish):
| Processor | Package |
-----------------------
| tokenize  | lfg     |
| pos       | lfg     |
| lemma     | lfg     |
| deppars

In [72]:
def get_sentiment1(statement):
    doc = nlp(statement)
    
    sentence_sentiment = 0

    for sentence in doc.sentences:
        for word in sentence.words:
            sent = 0
            val = sentiment_dictionary.category[sentiment_dictionary.word == word.lemma]
            if(val.empty):
                val = ''
            else:    
                val = val.values[0]

            if(val == 'H'):
                sentence_sentiment +=1 
                sent = 1
            elif(val == 'A' or val == 'S' or val == 'F' or val == 'D'):
                sentence_sentiment -= 1
                sent == -1

            print(word.text, sent)

    print("Sentyment zdania to: "+str(sentence_sentiment))
    
def get_sentiment(statement):
    doc = nlp(statement)
    
    text_sentiment = 0
    for sentence in doc.sentences:
        sentence_sentiment = 0
        for word in sentence.words:
            sent = 0
            val = sentiment_dictionary.sentiment[sentiment_dictionary.word == word.lemma]
            if(val.empty):
                sent = 0
            else:    
                sent = int(val.values[0])

            sentence_sentiment += sent 
            print(word.text, sent)
        text_sentiment += sentence_sentiment
        
    if(text_sentiment>=0.33):
        return 1
    elif text_sentiment<=-0.33:
        return -1
    else:
        return 0

**Informacje o zdaniu**

In [28]:
def get_info(statement):
    doc = nlp(statement)
    
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.lemma, word.pos)
            
    for sentence in doc.sentences:
        print(sentence.ents)
        print(sentence.dependencies)
        
    print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

***Tests***

In [75]:
get_sentiment("Lody są złe.")

Lody 0
są 0
złe -1
. 0


-1

In [61]:
#Lemmatise csv and ovveride it. Input and output file has two columns ['word', 'sentiment']
def lemmatise_csv(file):
    sentiment_dictionary_raw = pd.read_csv(file)
    sentiment_dictionary_lemma = pd.DataFrame(columns=['word', 'sentiment'] )
    for i in range(len(sentiment_dictionary_raw['word']) ):
        word = sentiment_dictionary_raw['word'][i]
        lemma = nlp(word).sentences[0].words[0].lemma
        sentiment = sentiment_dictionary_raw['sentiment'][i]
        sentiment_dictionary_lemma = sentiment_dictionary_lemma.append({'word':lemma, 'sentiment':sentiment}, ignore_index=True)
    sentiment_dictionary_lemma.to_csv(file)
    return sentiment_dictionary_lemma